In [1]:
from openai import OpenAI
import os
from dotenv import dotenv_values
from ReadLoad import read_jsonl, write_jsonl, read_json,write_json
from tqdm import tqdm
from evaluation import get_embedding, cosine_similarity
from prompt_template import get_sys_prompt, get_qa_prompt
#from shusheng import  get_respond
from qwen import qwen_response
import pandas as pd
import time

config = dotenv_values('.env')
client = OpenAI(
    api_key=config['qwen_key'],
    base_url=config['qwen_url']
)


def get_response(prompt):
    time.sleep(0.1)
    try:
        completion = client.chat.completions.create(
            model="qwen2-7b-instruct",
            messages=[
                {'role': 'system', 'content': get_sys_prompt()},
                {'role': 'user', 'content': prompt}
            ]

            )
        return completion.choices[0].message.content
    except Exception as e:
    # Handle any other unexpected exceptions
        print(f"An unexpected error occurred: {e}")
        return f"An unexpected error occurred: {e}"


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-18 23:44:18.198962: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-18 23:44:18.241319: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-18 23:44:19.623586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loadi

In [2]:
def completion(data):
    for d in tqdm(data):
        query = d['问题']
        clause = d['条款']
        name = d['产品名']
        prompt = get_qa_prompt(name,clause,query)
        d['prompt'] = prompt
        #d['answer'] = get_response(prompt)
        #d['answer'] = get_respond(prompt)
        d['answer'] = qwen_response(prompt)
    return data

In [3]:
def evaluation(data, name="测试数据"):
    similaritys = []
    for d in tqdm(data):
        vec1 = dev_data_embd[d['ID']]
        vec2 = get_embedding(d['answer'])
        similarity = cosine_similarity(vec1, vec2)
        d['similarity'] = similarity
        similaritys.append(similarity)
    length = len(data)
    score = sum(similaritys)/length
    print(f'测评数据集：{length} 模型推理结果与答案的相识度得分：{score}')
    df = pd.DataFrame(data)
    df.to_excel(f"{name}_score_{score}.xlsx", index=False)
    return data

In [4]:
def to_summit_json(data, commit="大道至简_result"):
    result = []
    for d in data:
        rd = {
            "ID": d['ID'],
            "question": d['问题'],
            "answer": d['answer']
        }
        result.append(rd)
    #write_json(result, commit)
    write_jsonl(result, commit)

### loading data

In [5]:
test_data = read_json("dataset/test.json")
# dev_data = read_jsonl("dataset/resultdev_with_embedding.jsonl")
# dev_data_embd = {}
# for d in dev_data:
#     dev_data_embd[d['ID']] = d['ans_embedding']
#     d.pop('ans_embedding')

In [6]:
# 将数据也移到GPU上
# comp_data = completion(dev_data[:100])
# eval_data = evaluation(comp_data)

test_data_result = completion(test_data)
to_summit_json(test_data_result)


100%|██████████| 1000/1000 [25:26<00:00,  1.53s/it]


In [7]:
from qwen import qwen_response
qwen_response("你好")

'你好！有什么关于保险条款的问题我可以帮助你解答吗？'

### 参考资料

https://dashscope.console.aliyun.com/billing